In [48]:
from bs4 import BeautifulSoup as bs 
import requests
import pandas as pd
import numpy as np
from time import sleep
import pprint

In [49]:
#WITHOUT HEADERS WE WILL GET ERROR 403 
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
#LISTS VALUES FROM SCRAPE
movie_name = []
year = []
duration = [] 
rate =[]
picture_url = []
category = []
director = []

In [50]:
#movie_name,year,duration,category,rate,director,actors,picture_url
pages=[1,2,3,4,5,6,7,8,9,10]
for num_page in pages:
    imdb_movies_url = 'https://www.imdb.com/list/ls048276758/?sort=moviemeter,asc&st_dt=&mode=detail&page='+str(num_page)
    page = requests.get(imdb_movies_url,headers=headers)
    soup = bs(page.content,'html.parser')
    data_per_movie = soup.findAll('div', attrs = {'class': 'lister-item mode-detail'})
    #Must put sleep - without the request may crash
    sleep(4)
   
    for store in data_per_movie:
        if store.h3.a.text is not None:
            name = store.h3.a.text
            movie_name.append(name)
        else: movie_name.append(None)


        if store.h3.find('span', class_ = "lister-item-year text-muted unbold") is not None:
            year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text.replace('(','').replace(')','')
            year.append(year_of_release)
        else:year.append(None)

        if store.p.find('span', class_ = 'runtime') is not None:            
            time = int(store.p.find('span', class_ = 'runtime').text.replace(' min',''))
            duration.append(time)
        else: duration.append(None)

        if store.p.find('span', class_ = 'genre') is not None:     
            categories = store.p.find('span', class_ = 'genre').text.replace("\n", "").rstrip().split(',')
            category.append(categories)
        else:category.append(None)

        if store.find('span', class_ = "ipl-rating-star__rating") is not None:
            rating = float(store.find('span', class_ = "ipl-rating-star__rating").text)
            rate.append(rating)
        else:rate.append(None)

        #Success only the first director
        if store.findAll('p',class_ = 'text-muted text-small')[1] is not None:
            directors_sections = store.findAll('p',class_ = 'text-muted text-small')[1]
            if directors_sections.find('a') is not None:
                director_name = directors_sections.find('a').text
                director.append(director_name)
            else:director.append(None)
        else:director.append(None)
        
      #TRY-EXCEPT not suit here because we need to fill the values per section   
  
  


In [61]:
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie Name": movie_name,\
                            "Year of Release" : year,\
                            "Duration (Min)": duration,\
                            "Genres": category,\
                            "Director" : director,\
                            "Movie Rating": rate
                            })

In [69]:
movie_list.describe()
movie_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Movie Name       1000 non-null   object 
 1   Year of Release  1000 non-null   object 
 2   Duration (Min)   1000 non-null   int64  
 3   Genres           999 non-null    object 
 4   Director         1000 non-null   object 
 5   Movie Rating     1000 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 47.0+ KB


In [64]:
# save the data in csv format
movie_list.to_csv("movies.csv")